## Part 3 Implementing & Improving AlexNet

Importing Required Libraries

In [17]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torchvision
from torchvision.io import read_image
from torchvision import transforms
from torch.utils.data import Dataset

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [21]:
print(torch.cuda.is_available())
device = torch.device('cuda')
device

True


device(type='cuda')

Step 1:

Loading the Data

In [5]:
dataset = []
for i in range(1, 10001):
    img = torchvision.io.read_image(f'C:/Users/RaghulKrish/Desktop/UB/Spring 23/ML/Assignment/A_2/cnn_dataset/dogs/{i}.jpg')
    dataset.append(img)
for i in range(1, 10001):
    img = torchvision.io.read_image(f'C:/Users/RaghulKrish/Desktop/UB/Spring 23/ML/Assignment/A_2/cnn_dataset/food/{i}.jpg')
    dataset.append(img)
for i in range(1, 10001):
    img = torchvision.io.read_image(f'C:/Users/RaghulKrish/Desktop/UB/Spring 23/ML/Assignment/A_2/cnn_dataset/vehicles/{i}.jpg')
    dataset.append(img)

In [6]:
labels = []
for i in range(1, 10001):
    labels.append(0)
for i in range(1, 10001):
    labels.append(1)
for i in range(1, 10001):
    labels.append(2)

In [18]:
data_path = "C:/Users/RaghulKrish/Desktop/UB/Spring 23/ML/Assignment/A_2/cnn_dataset/"
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

data = torchvision.datasets.ImageFolder(root = data_path, transform = TRANSFORM_IMG)

In [20]:
img = torch.tensor(data[0][0])
data = data.to(device)

(tensor([[[ 2.1462,  2.1462,  2.1633,  ...,  2.2147,  2.2147,  2.2147],
         [ 2.1462,  2.1462,  2.1633,  ...,  2.2147,  2.2147,  2.2147],
         [ 2.1633,  2.1633,  2.1804,  ...,  2.2147,  2.2147,  2.2147],
         ...,
         [-0.1143, -0.1143, -0.1314,  ...,  0.0912,  0.0741,  0.0741],
         [-0.0972, -0.0972, -0.1143,  ...,  0.0741,  0.0569,  0.0569],
         [-0.0972, -0.0972, -0.1143,  ...,  0.0741,  0.0569,  0.0569]],

        [[ 2.4111,  2.4111,  2.4111,  ...,  2.3936,  2.3936,  2.3936],
         [ 2.4111,  2.4111,  2.4111,  ...,  2.3936,  2.3936,  2.3936],
         [ 2.3936,  2.3936,  2.3936,  ...,  2.3936,  2.3936,  2.3936],
         ...,
         [-1.5980, -1.5980, -1.6331,  ..., -1.8606, -1.8782, -1.8782],
         [-1.5805, -1.5805, -1.6155,  ..., -1.8782, -1.8957, -1.8957],
         [-1.5805, -1.5805, -1.6155,  ..., -1.8782, -1.8957, -1.8957]],

        [[ 2.5006,  2.5006,  2.5006,  ...,  2.6051,  2.6051,  2.6051],
         [ 2.5006,  2.5006,  2.5006,  ...,  

C:\Users\RaghulKrish\AppData\Local\Temp\ipykernel_6016\4191424641.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(data[0][0])


Step 2

In [5]:
# Alex Net
class AlexNet(nn.Module):
    def __init__(self, no_of_class = 3):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size = 11, stride = 4, padding = 0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc3 = nn.Sequential(
            nn.Linear(4096, no_of_class))
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x